In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore') 

# Update rcParams in mpl
rcParams = {
        'font.size': 20,                # all fonts-size in plot
        'font.weight': 'bold',          # bold all fonts
        'figure.titleweight': 'bold',   # bold supertitle
        'axes.linewidth' : 6,
        'xtick.major.width': 6,
        'axes.spines.top': True,
        'axes.spines.right': True,
        'lines.linewidth': 6,
        'legend.fontsize': 'large',
        'xtick.labelsize': 'large',
        'ytick.labelsize': 'large',
        'xtick.labelsize': 20,   # sets x-tick font size
        'ytick.labelsize': 20,
        'axes.titlepad': 20,   # spacing between suptitle and figure
        'axes.facecolor': 'white'
    }

# import Adhoc module
from CycloidConnector import VWCycloid 

# Lets initialize VWCycloid class
redshift = VWCycloid() 

## Time at each recipe step

In [78]:
data = pd.read_csv("timespan_recipe.csv", usecols = [
'doe', 'instance', 'cycle_num', 'charging_state_name', 'elapsed_minutes','cell1_voltage_v',
'cell2_voltage_v', 'cell3_voltage_v', 'cell4_voltage_v', 'cell5_voltage_v', 'avg_current_a',
'avail_battery_capacity_ah', 'avg_voltage_v'
]).query("charging_state_name == 'CHARGE'").reset_index(drop = True)

In [83]:
data.cycle_num.unique()

array([2, 3, 4, 5])

In [103]:
def get_step_time(df, current_threshold = 0.6, sigma_multiplier = 3.0, window = 300):
    # Copy the df
    data = df.copy()
    
    # Find the time to CC state
    voltages = [f'cell{i}_voltage_v' for i in range(1, 6)]
    conditions = (data[voltages] >= 4.20).any(axis=1)
    first_index = data.index[conditions][:50]
    cc_time = data.loc[first_index, 'elapsed_minutes'].max()
    cv_time = data['elapsed_minutes'].max() - cc_time

    # Cut off data before CV period starts
    df = df[df['elapsed_minutes'].between(0, cc_time)].drop(columns=voltages)\
        .dropna(subset=['avg_current_a']).reset_index(drop=True)

    # Make sure initial currents are close to 0 (or what it should be)
    df.loc[df.elapsed_minutes < 1.0, 'avg_current_a'] = df[df.elapsed_minutes < 1.0].avg_current_a.median()

     # Create a desired result DataFrame
    result = pd.DataFrame()
    doe = df['doe'].unique()[0]
    cycle_num = df['cycle_num'].unique()[0]
    instance = df['instance'].unique()[0]
    charging_state_name = ['CHARGE']

    # Since we need to handle outliers and avg_current_a contain np.nan values

    def outlier_handler(series: pd.Series, sigma_multiplier : float = sigma_multiplier, window = window) -> pd.Series:
        """
        Handle outliers in a pandas Series.

        Parameters:
            series (pd.Series): The input Series.
            sigma_multiplier (float): Defines the boundary for outliers.

        Returns:
            pd.Series: Series with outliers handled.
        """
        roll = series.rolling(window = window, min_periods = 1)
        md = roll.median()
        sd = roll.std(ddof = 0)
        z_score = (series - md) / sd
        series[(z_score < - sigma_multiplier) | (z_score > sigma_multiplier)] = md
        return series
    
    # Lets define a new DataFrame
    current_df = df[['elapsed_minutes', 'avg_current_a']].dropna().reset_index(drop = True)

    # Handle outliers in avg_current_a
    current_df['avg_current_a'] = outlier_handler(current_df['avg_current_a'])

    # Identify where the current step occurs
    current_df['delta_current'] = current_df['avg_current_a'].diff().abs()

    # Identify step changes
    step_changes = current_df[current_df['delta_current'] > current_threshold]

    # Get the original dataframe indices where steps change occur
    # Add first and last index as well
    indices = [0] + step_changes.index.to_list() + [len(df) - 1]

    m = len(indices) - 1

    result['doe'] = [doe] * m
    result['instance'] = [instance] * m
    result['cycle_num'] = [cycle_num] * m
    result['charging_state_name'] = charging_state_name * m
    result['step_avg_current_a'] = [current_df.loc[indices[i]:indices[i+1], 'avg_current_a'].median() for i in range(m)]
    result['step_time_length_min'] = [current_df.loc[indices[i]:indices[i+1], 'elapsed_minutes'].max() - current_df.loc[indices[i]:indices[i+1], 'elapsed_minutes'].min() for i in range(m)]

    # Calculate voltage and capacity exactly at the time frames
    time_steps = [0] + result['step_time_length_min'].cumsum().to_list()
    time_steps.append(data['elapsed_minutes'].max())

    # Get the exact voltage values at the time frames
    voltage_values = []
    voltage_values.append(data.loc[data['avg_voltage_v'].first_valid_index(), 'avg_voltage_v'])
    for i in range(len(time_steps) - 1):
        voltage_values.append(data[data['elapsed_minutes'].between(time_steps[i], time_steps[i + 1])]['avg_voltage_v'].max())

    voltage_ranges = [(voltage_values[i], voltage_values[i + 1]) for i in range(len(voltage_values) - 1)]

    # Get the exact capacity values at the time frames
    capacity_values = []
    capacity_values.append(data.loc[data['avail_battery_capacity_ah'].first_valid_index(), 'avail_battery_capacity_ah'])
    for i in range(len(time_steps) - 1):
        capacity_values.append(data[data['elapsed_minutes'].between(time_steps[i], time_steps[i + 1])]['avail_battery_capacity_ah'].max())

    capacity_ranges = [(capacity_values[i], capacity_values[i + 1]) for i in range(len(capacity_values) - 1)]

    result['min_avg_voltage_v'] = [voltage[0] for voltage in voltage_ranges[:-1]]
    result['max_avg_voltage_v'] = [voltage[1] for voltage in voltage_ranges[:-1]]
    result['min_capacity_ah'] = [capacity[0] for capacity in capacity_ranges[:-1]]
    result['max_capacity_ah'] = [capacity[1] for capacity in capacity_ranges[:-1]]

    # Add CV period data for completeness
    result.loc[len(result)] = [
        doe, instance, cycle_num, charging_state_name[0], 'cv_current', cv_time, 
        voltage_ranges[-1][0],
        voltage_ranges[-1][1],
        capacity_ranges[-1][0],
        capacity_ranges[-1][1]
    ]

    return result, current_df


In [104]:
store = []
instance_cycle = [(i, j) for i in range(36, 40) for j in range(2, 4)]

for instance, cycle in instance_cycle:
    test = data[(data['instance'] == instance) & (data['cycle_num'] == cycle)]
    times = get_step_time(test)[0]
    store.append(times)

result_df = pd.concat(store, axis = 0)

In [131]:
result_df.groupby(['doe', 'instance', 'cycle_num', 'charging_state_name'])['step_time_length_min'].sum().reset_index(name = 'charge time')

,doe,instance,cycle_num,charging_state_name,charge time
0,testPackFultonv670,36,2,CHARGE,67.409567
1,testPackFultonv670,36,3,CHARGE,68.814450
2,testPackFultonv670,37,2,CHARGE,68.562617
3,testPackFultonv670,37,3,CHARGE,62.833433
4,testPackFultonv670,38,2,CHARGE,60.520700
5,testPackFultonv670,38,3,CHARGE,60.324767
6,testPackFultonv670,39,2,CHARGE,57.662083
7,testPackFultonv670,39,3,CHARGE,56.281933


In [132]:
data[(data['charging_state_name'] == 'CHARGE') & (data['cycle_num'] < 4)].groupby(['doe', 'instance', 'cycle_num', 'charging_state_name'])['elapsed_minutes'].max().reset_index(name = 'charge time')

,doe,instance,cycle_num,charging_state_name,charge time
0,testPackFultonv670,36,2,CHARGE,67.411250
1,testPackFultonv670,36,3,CHARGE,68.822883
2,testPackFultonv670,37,2,CHARGE,68.567733
3,testPackFultonv670,37,3,CHARGE,62.838300
4,testPackFultonv670,38,2,CHARGE,60.529233
5,testPackFultonv670,38,3,CHARGE,60.336567
6,testPackFultonv670,39,2,CHARGE,57.669567
7,testPackFultonv670,39,3,CHARGE,56.287050


In [105]:
result_df.to_excel("test.xlsx", index = False)

In [118]:
test = data[(data['instance'] == 36) & (data['cycle_num'] == 2)]

In [88]:
# store.append(get_step_time(test))
x = []
x.append(get_step_time(test)[0])

In [113]:
data = pd.read_csv("time_span.csv").dropna(subset = 'i_mean_a')

In [119]:
test = data.query("instance == 36 & cycle_num == 2 & charging_state_name == 'CHARGE'")

In [121]:
%matplotlib tk
sns.scatterplot(data = test, x = 'elapsed_minutes', y = 'i_mean_a')

<Axes: xlabel='elapsed_minutes', ylabel='i_mean_a'>